**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Apr 15 23:27:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14924, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 14924 (delta 21), reused 30 (delta 12), pack-reused 14880
Receiving objects: 100% (14924/14924), 13.37 MiB | 25.07 MiB/s, done.
Resolving deltas: 100% (10139/10139), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "night" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-15 23:28:34--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  43.7MB/s    in 3.9s    

2021-04-15 23:28:38 (39.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/__MACOSX/._classes.txt  
  inflating: data/obj/Movie on 2021-4-15 at 11.30 #2_000.jpg  
  inflating: data/obj/__MACOSX/._Movie on 2021-4-15 at 11.30 #2_000.jpg  
  inflating: data/obj/Movie on 2021-4-15 at 11.30 #2_000.txt  
  inflating: data/obj/Movie on 2021-4-15 at 11.30 #2_001.jpg  
  inflating: data/obj/__MACOSX/._Movie on 2021-4-15 at 11.30 #2_001.jpg  
  inflating: data/obj/Movie on 2021-4-15 at 11.30 #2_001.txt  
  inflating: data/obj/Movie on 2021-4-15 at 11.30 #2_003.jpg  
  inflating: data/obj/__MACOSX/._Movie on 2021-4-15 at 11.30 #2_003.jpg  
  inflating: data/obj/Movie on 2021-4-15 at 11.30 #2_003.txt  
  inflating: data/obj/Movie on 2021-4-15 at 11.30 #2_004.jpg  
  inflating: data/obj/__MACOSX/._Movie on 2021-4-15 at 11.30 #2_004.jpg  
  inflating: data/obj/Movie on 2021-4-15 at 11.30 #2_004.txt  
  inflating: data/obj/Movie on 2021-4-15 at 11.30 #2_006.jpg  
  inflating: da

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

2 0.254167 0.567361 0.262037 0.306944

['2', '0.254167', '0.567361', '0.262037', '0.306944']
0 data/obj/Movie on 2021-4-15 at 11.34_026.txt
0 0.254167 0.567361 0.262037 0.306944
2 0.171759 0.525000 0.195370 0.297222

['2', '0.171759', '0.525000', '0.195370', '0.297222']
1 data/obj/Movie on 2021-4-15 at 11.30 #2_019.txt
0 0.171759 0.525000 0.195370 0.297222
2 0.186574 0.545833 0.250926 0.327778

['2', '0.186574', '0.545833', '0.250926', '0.327778']
2 data/obj/Movie on 2021-4-15 at 11.34_000.txt
0 0.186574 0.545833 0.250926 0.327778
2 0.225926 0.568750 0.272222 0.340278

['2', '0.225926', '0.568750', '0.272222', '0.340278']
3 data/obj/Movie on 2021-4-15 at 11.34_049.txt
0 0.225926 0.568750 0.272222 0.340278
2 0.234722 0.579167 0.275000 0.333333

['2', '0.234722', '0.579167', '0.275000', '0.333333']
4 data/obj/Movie on 2021-4-15 at 11.34_043.txt
0 0.234722 0.579167 0.275000 0.333333
2 0.222222 0.591667 0.270370 0.325000

['2', '0.222222', '0.591667', '0.270370', '0.325000']
5 data/obj/Mov

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/Movie on 2021-4-15 at 11.34_009.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_029.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_043.jpg', 'data/obj/Movie on 2021-4-15 at 11.34_006.jpg', 'data/obj/Movie on 2021-4-15 at 11.34_026.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_023.jpg', 'data/obj/Movie on 2021-4-15 at 11.34_053.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_010.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_066.jpg', 'data/obj/Movie on 2021-4-15 at 11.34_029.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_020.jpg', 'data/obj/Movie on 2021-4-15 at 11.34_051.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_057.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_051.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_064.jpg', 'data/obj/Movie on 2021-4-15 at 11.30 #2_067.jpg', 'data/obj/Movie on 2021-4-15 at 11.34_054.jpg', 'data/obj/Movie on 2021-4-15 at 11.34_036.jpg', 'data/obj/Movie on 2021-4-15 at 11.34_049.jpg', 'data/obj/Movie on 2021-4-15 at 11.34_019.jpg', 'data/obj

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

流式输出内容被截断，只能显示最后 5000 行内容。
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.905141), count: 1, class_loss = 0.000000, iou_loss = 0.003044, total_loss = 0.003044 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 122932, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.875863), count: 4, class_loss = 0.131423, iou_loss = 0.038979, total_loss = 0.170403 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 122934, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (io